In [ ]:
self.datasetdscr = datasetdscr
import unified_ar.general.Cache as Cache
Cache.GlobalDisable = True
starts = s_events.time.dt.floor(self.meta_start_period).unique()
ends = starts+self.meta_size

meta_features = []
meta_targets = []
from constants import methods
tmp_segments = methods.segmentation
methods.segmentation = methods.meta_segmentation_sub_tasks
self.segmentor_dic = {p['method']().shortname(): p for p in methods.meta_segmentation_sub_tasks}
if 1:
import ml_strategy.Simple
import logging
logging.getLogger().setLevel(logging.INFO)
fast_strategy = ml_strategy.Simple.NormalStrategy()
fast_strategy.ui_debug = {'seg': 0}
for s, e in zip(starts, ends):
    data2 = self.customSplit(s_events, a_events, s, e)
    print(f's={s} : {e}============= #sevent={len(data2.s_events)} #aevents={len(data2.a_events)}')
    if len(data2.s_events) == 0 or len(data2.a_events) == 0:
        continue

    result = fast_strategy.train(datasetdscr, data2, acts, update_model=True)
    if result is None:
        continue
    logger.debug(fast_strategy.get_info().functions)
    # result=fast_strategy.pipeline(fast_strategy.functions,self.traindata,train=True)
    aggr = data2.s_events.groupby('SID').count()
    fea = {k: aggr.loc[k]['value'] if k in aggr.index else 0 for k in datasetdscr.sensor_id_map_inverse}
    fea['time'] = s
    meta_features.append(fea)
    seg = result.functions['segmentor']
    meta_targets.append({'method': seg[0], **result.quality, **{p: seg[1][p] for p in seg[1]}})
    # self.strategy=fast_strategy
methods.segmentation = tmp_segments
logging.getLogger().setLevel(logging.DEBUG)
d = {'meta_features': meta_features, 'meta_targets': meta_targets}
import unified_ar.general.utils
from constants import methods
general.utils.saveState(d, f"meta_dataset/{methods.run_names['out']} {methods.run_names['fold']}")
else:
import unified_ar.general.utils
d = general.utils.loadState(
    "meta_dataset 220602_23-52-11-A4H-Namespace(classifier=0, comment='0', dataset=3, evaluation=0, feature_extraction=0, mlstrategy=0, output='logs', segmentation=0) 0")

feat_df = pd.DataFrame(d['meta_features'])
target_df = pd.DataFrame(d['meta_targets'])
# print(feat_df.describe())
ntarget = target_df.drop(['accuracy', 'precision', 'recall', 'f1'], axis=1)

self.targetTransformer = MyTargetTransformer(self.meta_mode)
self.targetTransformer.fit(ntarget)
self.featTransformer = self.create_feat_transformer(feat_df)
self.featTransformer.fit(feat_df)
print(f"metainfo=============== \nfeat={feat_df}\n target={target_df}")
X = self.featTransformer.transform(feat_df)
y = self.targetTransformer.transform(ntarget)
if self.meta_mode == 'keras':
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(X.shape[1],))
layer1 = layers.Dense(16, activation='relu')(inputs)
layer2 = layers.Dense(8, activation='relu')(layer1)
layer3 = layers.Dense(16, activation='relu')(layer2)
classifier = layers.Dense(1, activation='softmax', name='method')(layer3)
regressions = [layers.Dense(1, activation='linear', name=x)(layer3) for x in ntarget.columns.drop('method')]

mdl = keras.Model(inputs=inputs, outputs=[classifier, *regressions])

mdl.compile(loss=['categorical_crossentropy', *(['mse']*len(regressions))], optimizer='adam', metrics=['accuracy'])
else:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
mdl = MultiOutputRegressor(SVR())
mdl.fit(X, y)
self.ml = mdl